In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


from functions import *

pd.set_option('display.max_columns', 100)

raw

In [2]:
app_train = pd.read_csv('dseb-63-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_application_train.csv').sort_values('SK_ID_CURR').reset_index(drop=True)
app_train.drop(['Unnamed: 0'], axis=1, inplace=True)
app_train.set_index('SK_ID_CURR', inplace=True)
app_train.head(10)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,...,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,Cash loans,M,N,Y,0,202500.0,1197000.0,44487.0,1197000.0,Unaccompanied,Working,Incomplete higher,Single / not married,House / apartment,0.026392,-11945,-376,-574.0,-580,NaN,1,1,0,1,0,0,NaN,1.0,2,2,WEDNESDAY,11,0,0,0,0,0,0,Other,0.284570,0.126697,0.285180,0.2454,NaN,0.9901,NaN,NaN,0.24,0.2069,...,0.2477,NaN,0.9901,NaN,NaN,0.24,0.2069,0.3750,NaN,NaN,NaN,0.2527,NaN,0.0470,NaN,block of flats,0.2053,Panel,No,7.0,0.0,7.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Cash loans,F,N,N,0,175500.0,900000.0,26316.0,900000.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.003122,-19158,-9203,-12984.0,-2568,NaN,1,1,0,1,0,0,Accountants,2.0,3,3,THURSDAY,12,0,0,0,0,0,0,School,NaN,0.598301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-142.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,0,Cash loans,F,N,Y,1,58500.0,265851.0,11263.5,229500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.031329,-14434,-3759,-4976.0,-3989,NaN,1,1,0,1,0,0,NaN,3.0,2,2,WEDNESDAY,9,0,0,0,0,0,0,School,0.439261,0.293988,0.415347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,Cash loans,F,N,N,2,54000.0,545040.0,20547.0,450000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.004849,-15957,-6018,-10110.0,-5219,NaN,1,1,0,1,1,0,Cleaning staff,4.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 2,NaN,0.070575,0.397946,0.1186,0.1234,0.9816,0.762,0.0331,0.08,0.1552,...,0.1197,0.1234,0.9816,0.7652,0.0333,0.08,0.1552,0.2500,0.0417,0.1153,0.1206,0.1064,0.0116,0.0147,org spec account,block of flats,0.0755,"Stone, brick",No,0.0,0.0,0.0,0.0,-725.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,2.0,2.0
4,0,Cash loans,F,Y,Y,0,135000.0,512064.0,25033.5,360000.0,Unaccompanied,Working,Secondary / 

eda

In [3]:
app_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246009 entries, 0 to 307510
Columns: 121 entries, TARGET to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 229.0+ MB


In [4]:
# check missing values
pd.Series(app_train.isnull().sum() / len(app_train)).sort_values(ascending=False)

COMMONAREA_AVG                0.698877
COMMONAREA_MODE               0.698877
COMMONAREA_MEDI               0.698877
NONLIVINGAPARTMENTS_AVG       0.694564
NONLIVINGAPARTMENTS_MODE      0.694564
                                ...   
REGION_POPULATION_RELATIVE    0.000000
NAME_HOUSING_TYPE             0.000000
NAME_FAMILY_STATUS            0.000000
NAME_EDUCATION_TYPE           0.000000
TARGET                        0.000000
Length: 121, dtype: float64

In [5]:
# drop columns with more than 50% missing values
cols_missing = app_train.columns[app_train.isnull().sum() / len(app_train) > 0.5]
app_train.drop(cols_missing, axis=1, inplace=True)
len(cols_missing)

41

In [6]:
# # plot distribution of each feature hue by TARGET
# for col in app_train.select_dtypes('object').columns:
#     plt.figure(figsize=(12, 6))
#     val0 = app_train.loc[app_train['TARGET'] == 0, col].value_counts(normalize=True)
#     val1 = app_train.loc[app_train['TARGET'] == 1, col].value_counts(normalize=True)
#     width = 0.35
#     sns.barplot(x=val0.index, y=val0.values, color='blue', alpha=0.75, label='0')
#     sns.barplot(x=val1.index, y=val1.values, color='red', alpha=0.75, label='1')
#     plt.title(col)
#     plt.legend()
#     plt.show()


In [7]:
# nunique categorical
app_train.select_dtypes('object').apply(pd.Series.unique, axis = 0)

NAME_CONTRACT_TYPE                                [Cash loans, Revolving loans]
CODE_GENDER                                                         [M, F, XNA]
FLAG_OWN_CAR                                                             [N, Y]
FLAG_OWN_REALTY                                                          [Y, N]
NAME_TYPE_SUITE               [Unaccompanied, Family, Spouse, partner, Child...
NAME_INCOME_TYPE              [Working, State servant, Commercial associate,...
NAME_EDUCATION_TYPE           [Incomplete higher, Higher education, Secondar...
NAME_FAMILY_STATUS            [Single / not married, Married, Separated, Civ...
NAME_HOUSING_TYPE             [House / apartment, Office apartment, Co-op ap...
OCCUPATION_TYPE               [nan, Accountants, Cleaning staff, Medicine st...
WEEKDAY_APPR_PROCESS_START    [WEDNESDAY, THURSDAY, TUESDAY, SATURDAY, SUNDA...
ORGANIZATION_TYPE             [Other, School, Business Entity Type 2, Univer...
EMERGENCYSTATE_MODE                     

In [8]:
# # normalized count plot for numerical features
# for col in app_train.select_dtypes('number').columns:
#     if col != 'TARGET':
#         plt.figure(figsize=(12, 6))
#         sns.kdeplot(data=app_train, x=col, hue='TARGET', common_norm=False)
#         plt.title(col)
#         plt.show()

feature selection

In [9]:
len(app_train.columns)

80

In [10]:
cols_to_keep = ['TARGET', 
                'CODE_GENDER', 
                'FLAG_OWN_CAR', 
                'CNT_CHILDREN',
                'AMT_CREDIT',
                'AMT_ANNUITY', 
                'AMT_GOODS_PRICE', 
                'NAME_INCOME_TYPE',
                'NAME_EDUCATION_TYPE', 
                'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
                'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
                'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 
                'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                'OCCUPATION_TYPE', 
                'REGION_RATING_CLIENT',
                'REGION_RATING_CLIENT_W_CITY',
                'REG_REGION_NOT_LIVE_REGION',
                'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
                'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
                'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 
                'EXT_SOURCE_2','EXT_SOURCE_3',
                'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_2',
                'FLAG_DOCUMENT_3', 
                'FLAG_DOCUMENT_21',
                'AMT_REQ_CREDIT_BUREAU_DAY', 
                'AMT_REQ_CREDIT_BUREAU_QRT']

app_train = app_train[cols_to_keep]
app_train.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_QRT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,M,N,0,1197000.0,44487.0,1197000.0,Working,Incomplete higher,Single / not married,House / apartment,0.026392,-11945,-376,-574.0,-580,1,0,0,0,NaN,2,2,0,0,0,0,0,0,Other,0.126697,0.285180,0.0,0,1,0,0.0,0.0
1,1,F,N,0,900000.0,26316.0,900000.0,State servant,Higher education,Married,House / apartment,0.003122,-19158,-9203,-12984.0,-2568,1,0,0,0,Accountants,3,3,0,0,0,0,0,0,School,0.598301,NaN,-142.0,0,1,0,0.0,0.0
2,0,F,N,1,265851.0,11263.5,229500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,0.031329,-14434,-3759,-4976.0,-3989,1,0,0,0,NaN,2,2,0,0,0,0,0,0,School,0.293988,0.415347,0.0,0,1,0,0.0,1.0
3,0,F,N,2,545040.0,20547.0,450000.0,Working,Secondary / secondary special,Married,House / apartment,0.004849,-15957,-6018,-10110.0,-5219,1,0,1,0,Cleaning staff,2,2,0,0,0,0,0,0,Business Entity Type 2,0.070575,0.397946,-725.0,0,1,0,0.0,2.0
4,0,F,Y,0,512064.0,25033.5,360000.0,Working,Secondary / secondary special,Married,House / apartment,0.018801,-17851,-495,-43.0,-181,1,0,0,0,Medicine staff,2,2,0,0,0,0,1,1,University,0.501790,0.529890,0.0,0,1,0,0.0,0.0


In [11]:
app_train.isnull().sum().sum()

193116

In [12]:
# handle missing values
nan_values = ['XNA', 'Unknown']
app_train.replace(nan_values, np.nan, inplace=True)

# keep 3 most frequent values for categorical features
for col in app_train.select_dtypes('object').columns:
    if col != 'TARGET' and app_train[col].nunique() > 4:
        top3 = app_train[col].value_counts().index[:3]
        app_train.loc[~app_train[col].isin(top3), col] = 'Other'

In [13]:
for col in app_train.select_dtypes('object').columns:
    print(col, app_train[col].unique())

CODE_GENDER ['M' 'F' nan]
FLAG_OWN_CAR ['N' 'Y']
NAME_INCOME_TYPE ['Working' 'Other' 'Commercial associate' 'Pensioner']
NAME_EDUCATION_TYPE ['Incomplete higher' 'Higher education' 'Secondary / secondary special'
 'Other']
NAME_FAMILY_STATUS ['Single / not married' 'Married' 'Other' 'Civil marriage']
NAME_HOUSING_TYPE ['House / apartment' 'Other' 'Municipal apartment' 'With parents']
OCCUPATION_TYPE ['Other' 'Laborers' 'Sales staff' 'Core staff']
ORGANIZATION_TYPE ['Other' 'Business Entity Type 3' 'Self-employed']


In [14]:
# fill nan values with mode for all features
for col in app_train.columns:
    app_train[col].fillna(app_train[col].mode()[0], inplace=True)
app_train.isnull().sum().sum()


0

In [15]:
# Check for nulls before transformation
print("Null values before transformation:", app_train.isnull().sum().sum())

onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoded = onehot_encoder.fit_transform(app_train.select_dtypes('object'))
onehot_encoded = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out())

# Ensure that indices are aligned
onehot_encoded.index = app_train.index

app_train.drop(app_train.select_dtypes('object').columns, axis=1, inplace=True)
app_train = pd.concat([app_train, onehot_encoded], axis=1)

# Check for nulls after transformation
print("Null values after transformation:", app_train.isnull().sum().sum())

Null values before transformation: 0
Null values after transformation: 0


In [16]:
# scale numerical features
scaler = MinMaxScaler()
app_train_scaled = pd.DataFrame(scaler.fit_transform(app_train), columns=app_train.columns)
app_train_scaled.head()

,TARGET,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_QRT,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Other,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Other,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Other,NAME_FAMILY_STATUS_Single / not married,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Other,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Self-employed
0,0.0,0.000000,0.287640,0.167199,0.288440,0.361433,0.748816,0.045767,0.975487,0.919411,1.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.148184,0.317876,1.000000,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.000000,0.213483,0.096332,0.214366,0.039215,0.342221,0.022730,0.445507,0.643185,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.699768,0.832817,0.966915,0.0,1.0,0.0,0.0,0.000000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.052632,0.055144,0.037627,0.047138,0.429796,0.608512,0.036938,0.787496,0.445741,1.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.343845,0.463236,1.000000,0.0,1.0,0.0,0.0,0.003831,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.105263,0.124854,0.073833,0.102132,0.063128,0.522661,0.031042,0.568244,0.274837,1.0,0.0,1.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.082543,0.443804,0.831081,0.0,1.0,0.0,0.0,0.007663,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.000000,0.116620,0.091330,0.079686,0.256321,0.415896,0.045457,0.998164,0.974851,1.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,1.0,1.0,0.586889,0.591148,1.000000,0.0,1.0,0.0,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [17]:
app_train.isnull().sum()

TARGET                                               0
CNT_CHILDREN                                         0
AMT_CREDIT                                           0
AMT_ANNUITY                                          0
AMT_GOODS_PRICE                                      0
REGION_POPULATION_RELATIVE                           0
DAYS_BIRTH                                           0
DAYS_EMPLOYED                                        0
DAYS_REGISTRATION                                    0
DAYS_ID_PUBLISH                                      0
FLAG_EMP_PHONE                                       0
FLAG_WORK_PHONE                                      0
FLAG_PHONE                                           0
FLAG_EMAIL                                           0
REGION_RATING_CLIENT                                 0
REGION_RATING_CLIENT_W_CITY                          0
REG_REGION_NOT_LIVE_REGION                           0
REG_REGION_NOT_WORK_REGION                           0
LIVE_REGIO

In [18]:
X, y = app_train_scaled.drop('TARGET', axis=1), app_train_scaled['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

logreg = LogisticRegression(class_weight='balanced', solver='newton-cholesky')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

gini(y_test, y_pred_proba)

0.46644627529726757

In [19]:
app_train.to_csv('processed-data/app_train.csv')